##### Note: The summaries for the other layers will come soon in the same format after the clean up is done. 

# Summary of best models for given numbers of layers

In [ ]:
# Each row is the output for a model in the following format. Different lines for different model structures. See the cell below for the excecuted codes.
# print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')
# Example: [16layers, 12(6)heads, hidden-dim=696, inter-layer-dim = 696x4, head-dim=224, #param=218million, train_loss, val_loss, Num-steps]

[+36, 4, 2, 648, 2592, 160, 259.0, 2.4992,+2.4858, 144000], # minGemma-hidden_layers36-att_heads4-kv_heads2-hidden648-intermediate2592-head_dim160-T512--2025-07-02-14-17.pth
[+30, 4, 2, 648, 2592, 224, 236.2, 2.5071,+2.4812, 144000], # minGemma-hidden_layers30-att_heads4-kv_heads2-hidden648-intermediate2592-head_dim224-T512--2025-06-18-19-30.pth
[+24, 6, 3, 648, 2592, 272, 229.8, 2.5135,+2.4748, 144000], # minGemma-hidden_layers24-att_heads6-kv_heads3-hidden648-intermediate2592-head_dim272-T512--2025-07-13-19-45.pth

[+18, 8, 4, 576, 2304, 256, 164.4, 2.5319,+2.4691, 288000], # minGemma-hidden_layers18-att_heads8-kv_heads4-hidden576-intermediate2304-head_dim256-T512--2025-07-21-10-59.pth
[+16, 9, 3, 612, 2448, 208, 151.7, 2.5393,+2.4714, 288000], # minGemma-hidden_layers16-att_heads9-kv_heads3-hidden612-intermediate2448-head_dim208-T512--2025-07-25-09-46.pth

[ 14, 8, 4, 704, 2816, 256, 179.3, 2.5348,+2.4772, 144000], # minGemma-hidden_layers14-att_heads8-kv_heads4-hidden704-intermediate2816-head_dim256-T512--2025-05-30-00-01.pth
[+12,12, 4, 672, 2688, 192, 148.4, 2.5476,+2.4740, 288000], # minGemma-hidden_layers12-att_heads12-kv_heads4-hidden672-intermediate2688-head_dim192-T512--2025-07-17-17-31.pth

# L36 (Models with 36 Layers)

In [ ]:
# L36 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)
[ 36, 5, 5, 640, 3840, 144, 364.3, 2.4763, 2.5109, 144000], # 41.2h 42.6G x6
[ 36, 5, 5, 640, 2560, 144, 275.7, 2.4952, 2.4890, 144000], # 36.5h 34.9G x4
[ 36, 4, 4, 600, 2400, 144, 235.7, 2.5155, 2.4896, 144000], # 32.8h 33.8G x4

[ 36, 4, 2, 696, 3480, 224, 364.2, 2.5042, 2.5060, 144000], # 43.4h 41.9G x5 TOO big # best normal model
[ 36, 4, 2, 696, 2784, 224, 311.9, 2.4954, 2.4920, 144000], # 40.4h 39.0G x4 TOO big
[ 36, 4, 2, 696, 2784, 192, 302.3, 2.4950, 2.5004, 144000], # 38.3h       x4 TOO big
[ 36, 4, 2, 696, 2784, 160, 292.6, 2.5039, 2.4934, 144000], # 36.5h       x4 TOO big
[ 36, 4, 2, 648, 2592, 224, 276.9, 2.4947, 2.4918, 144000], # 38.6h 35.1G x4 TOO big
[ 36, 4, 2, 648, 2592, 192, 268.0, 2.4897, 2.4893, 144000], # 36.8h       x4 TOO big
[+36, 4, 2, 648, 2592, 160, 259.0, 2.4992,+2.4858, 144000], # 35.0h       x4 (MIN confirmed)  minGemma-hidden_layers36-att_heads4-kv_heads2-hidden648-intermediate2592-head_dim160-T512--2025-07-02-14-17
[ 36, 4, 2, 648, 2592, 144, 254.5, 2.4979, 2.4899, 144000], # 34.3h       x4 TOO small
[ 36, 4, 2, 600, 2400, 160, 227.4, 2.5029, 2.4865, 144000], # 32.6h       x4 TOO small
[ 36, 4, 2, 600, 2400, 144, 223.2, 2.5039, 2.4899, 144000], # 32.0h       x4 TOO small

In [8]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  36 # 8 # G28 G18 #blocks
num_attention_heads = 4 # 4 # G16 G8
num_key_value_heads = 2 # 4 # G16 G1
hidden_size = num_attention_heads*162 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 160 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=28; N_step=144000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4858:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L36 att4 kv_heads2 hidden648 intermediate2592 head_dim160 T512


Step,Training Loss
144000,2.499200


[ 36, 4, 2, 648, 2592, 160, 259.0, 2.4992, 2.4858, 144000],


# L30

In [ ]:
# L30 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)

[ 30, 6, 3, 720, 2880, 160, 285.3, 2.4815, 2.4926, 144000], # 36.5h 36.3G
[ 30, 5, 5, 640, 2560, 144, 235.1, 2.5239, 2.4838, 144000], # 31.1h 33.2G (if x6 37.8G)
[ 30, 5, 1, 720, 2880, 288, 297.7, 2.4904, 2.4944, 144000], # 41.2h 34.7G # 2nd best

[ 30, 4, 2, 696, 2784, 224, 265.7, 2.5058, 2.4898, 144000], # 34.3h       x4 TOO big
[ 30, 4, 2, 648, 2592, 256, 243.7, 2.4909, 2.4829, 144000], # 34.2h       x4 TOO big
[+30, 4, 2, 648, 2592, 224, 236.2, 2.5071,+2.4812, 144000], # 32.7h 31.3G x4 (MIN confirmed) # best 10M normal # minGemma-hidden_layers30-att_heads4-kv_heads2-hidden648-intermediate2592-head_dim224-T512--2025-06-18-19-30.pth
[ 30, 4, 2, 648, 2592, 192, 228.7, 2.5032, 2.4832, 144000], # 31.3h 30.9G x4 TOO small
[ 30, 4, 2, 600, 2400, 224, 208.3, 2.5080, 2.4820, 144000], # 30.9h 29.8G x4 TOO small

In [14]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  30 # 8 # G28 G18 #blocks
num_attention_heads = 4 # 4 # G16 G8
num_key_value_heads = 2 # 4 # G16 G1
hidden_size = num_attention_heads*150 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 224 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=28; N_step=144000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4812:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L30 att4 kv_heads2 hidden600 intermediate2400 head_dim224 T512


Step,Training Loss
144000,2.508000


[ 30, 4, 2, 600, 2400, 224, 208.3, 2.5080, 2.4820, 144000],


# L24

In [ ]:
# L24 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)
[ 24, 8, 4, 704, 2816, 144, 236.7, 2.4969, 2.4841, 144000], # 32.0h 29.1G x4
[ 24, 6, 3, 696, 2784, 240, 246.8, 2.5042, 2.4846, 144000], # 34.4h 29.8G x4 TOO big
[ 24, 6, 3, 696, 2784, 272, 256.5, 2.4990, 2.4836, 144000], # 37.5h 33.1G x4 TOO big
[ 24, 6, 3, 648, 2592, 304, 238.8, 2.5055, 2.4799, 144000], # 38.3h 30.1G x4 TOO big
[+24, 6, 3, 648, 2592, 272, 229.8, 2.5135,+2.4748, 144000], # 36.5h 29.2G x4 (MIN confirmed) minGemma-hidden_layers24-att_heads6-kv_heads3-hidden648-intermediate2592-head_dim272-T512--2025-07-13-19-45.pth
[ 24, 6, 3, 648, 3888, 240, 281.4, 2.4856, 2.4908, 144000], # 36.4h 34.6G x6 TOO big
[ 24, 6, 3, 648, 3240, 240, 251.1, 2.5044, 2.4878, 144000], # 35.1h 30.3G x5 TOO big
[ 24, 6, 3, 648, 2592, 240, 220.9, 2.5138,+2.4805, 144000], # 33.2h       x4 TOO small # best normal
[ 24, 6, 3, 600, 2400, 272, 204.5, 2.5241, 2.4817, 144000], # 34.7h 28.6G x4 TOO small

In [16]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  24 # 8 # G28 G18 #blocks
num_attention_heads = 6 # 4 # G16 G8
num_key_value_heads = 3 # 4 # G16 G1
hidden_size = num_attention_heads*108 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 272 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=28; N_step=144000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4805:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L24 att6 kv_heads3 hidden648 intermediate2592 head_dim272 T512


Step,Training Loss
144000,2.513500


[ 24, 6, 3, 648, 2592, 272, 229.8, 2.5135, 2.4748, 144000],


# L18

In [ ]:
# L18 Normal (default: x4 B28 lr10e-4 WD0.25 N144000)
[ 18, 8, 4, 704, 4224, 256, 274.0, 2.5034, 2.4883, 144000], # 33.9h 30.0G x6 TOO big
[ 18, 8, 4, 704, 3520, 256, 247.2, 2.5168, 2.4852, 288000], # 37.9h 18.1G x5 TOO big
[ 18, 8, 4, 704, 2816, 256, 220.4, 2.5578, 2.4759, 144000], # 31.4h 26.9G x4 TOO big
[ 18, 8, 4, 704, 2816, 224, 210.7, 2.5325, 2.4750, 288000], # 33.9h       x4
[ 18, 8, 4, 640, 2560, 288, 200.4, 2.5795, 2.4741, 288000], # 36.0h 15.8G x4 TOO big
[ 18, 8, 4, 640, 2560, 256, 191.5, 2.5149, 2.4727, 288000], # 32.4h       x4 TOO big
[ 18, 8, 4, 640, 2560, 224, 182.7, 2.5207, 2.4751, 288000], # 30.8h       x4 TOO small
[ 18, 8, 4, 608, 2432, 256, 177.8, 2.5305, 2.4770, 288000], # 31.5h 15.2G x4 TOO big
[ 18, 8, 4, 576, 2304, 288, 172.4, 2.5352, 2.4774, 288000], # 34.8h 15.1G x4 TOO big
[+18, 8, 4, 576, 2304, 256, 164.4, 2.5319,+2.4691, 288000], # 30.9h 14.8G x4 (MIN confirmed)  minGemma-hidden_layers18-att_heads8-kv_heads4-hidden576-intermediate2304-head_dim256-T512--2025-07-21-10-59.pth
[ 18, 8, 4, 576, 2304, 224, 156.5, 2.5369, 2.4718, 288000], # 29.5h 14.5G x4 TOO small
[ 18, 8, 4, 544, 2176, 256, 151.5, 2.5400, 2.4835, 288000], # 30.3h 14.5G x4 TOO small

In [5]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  18 # 8 # G28 G18 #blocks
num_attention_heads = 8 # 4 # G16 G8
num_key_value_heads = 4 # 4 # G16 G1
hidden_size = num_attention_heads*72 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 256 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)
        
class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=32; B=28//2; N_step=144000*2; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(gradient_accumulation_steps=2, learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4727:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L18 att8 kv_heads4 hidden576 intermediate2304 head_dim256 T512


Step,Training Loss
144000,2.531900


[ 18, 8, 4, 576, 2304, 256, 164.4, 2.5319, 2.4691, 288000],


# L16

In [ ]:
# L16 Normal (default: x4 B28 lr10e-4 WD0.25 N144000)
[ 16, 9, 3, 720, 2880, 224, 197.8, 2.5124, 2.4812, 144000], # 28.1h       x4 TOO big
[ 16, 9, 3, 720, 2880, 192, 188.9, 2.5388, 2.4785, 288000], # 30.1h       x4 TOO big
[ 16, 9, 3, 720, 2880, 176, 184.5, 2.5426, 2.4773, 288000], # 29.4h 14.8G x4 TOO big
[ 16, 9, 3, 684, 4104, 224, 228.1, 2.4995, 2.4857, 144000], # 30.6h 27.3G x6 TOO big (x4 sufices)
[ 16, 9, 3, 684, 2736, 224, 183.2, 2.5218, 2.4788, 144000], # 27.8h 24.9G x4 TOO big
[ 16, 9, 3, 684, 2736, 208, 179.0, 2.5242, 2.4744, 288000], # 30.9h 14.6G x4 TOO big
[ 16, 9, 3, 684, 2736, 192, 174.8, 2.5343,+2.4720, 288000], # 29.6h       x4 (local MIN confirmed) (4th best)
[ 16, 9, 3, 684, 2736, 176, 170.6, 2.5343, 2.4741, 288000], # 28.9h 14.6G x4 TOO small
[ 16, 9, 3, 648, 2592, 224, 169.0, 2.5303, 2.4784, 144000], # 27.3h 24.0G x4 TOO big
[+16, 9, 3, 648, 2592, 208, 165.1, 2.5289,+2.4714, 288000], # 30.1h 14.6G x4 (MIN confirmed) BEST-tie  minGemma-hidden_layers16-att_heads9-kv_heads3-hidden648-intermediate2592-head_dim208-T512--2025-08-07-22-27.pth
[ 16, 9, 3, 648, 3240, 192, 181.3, 2.5101, 2.4812, 144000], # 26.5h 24.9G x5 TOO big
[ 16, 9, 3, 648, 2592, 192, 161.1, 2.5199, 2.4734, 144000], # 27.6h 23.3G x4
[ 16, 9, 3, 648, 2592, 176, 157.1, 2.5230, 2.4738, 288000], # 28.3h 14.3G x4 TOO small
[ 16, 9, 3, 612, 2448, 240, 159.2, 2.5341, 2.4791, 288000], # 30.2h       x4 TOO big
[ 16, 9, 3, 612, 2448, 224, 155.4, 2.5305,+2.4717, 288000], # 29.1h       x4 TOO big (3rd best)
[+16, 9, 3, 612, 2448, 208, 151.7, 2.5393,+2.4714, 288000], # 28.5h       x4 (MIN confirmed) BEST-tie  minGemma-hidden_layers16-att_heads9-kv_heads3-hidden612-intermediate2448-head_dim208-T512--2025-07-25-09-46.pth
[ 16, 9, 3, 612, 2448, 192, 147.9,23.4609,    nan, 288000], # 26.2h       x4 (failed, trying again)
[ 16, 9, 3, 612, 3060, 176, 162.1, 2.5303, 2.4759, 288000], # 27.9h 14.5G x5 TOO big
[ 16, 9, 3, 612, 2448, 176, 144.1, 2.5350, 2.4741, 144000], # 25.4h 22.8G
[ 16, 9, 3, 576, 2304, 224, 142.3, 2.5372, 2.4761, 288000], # 27.8h       x4 TOO small
[ 16, 9, 3, 576, 2304, 208, 138.8, 2.5390, 2.4771, 288000], # 26.9h 13.9G x4 TOO small

In [19]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  16 # 8 # G28 G18 #blocks
num_attention_heads = 9 # 4 # G16 G8
num_key_value_heads = 3 # 4 # G16 G1
hidden_size = num_attention_heads*72 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 208 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)
        
class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=32; B=28//2; N_step=144000*2; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(gradient_accumulation_steps=2, learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4717:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L16 att9 kv_heads3 hidden648 intermediate2592 head_dim208 T512


Step,Training Loss
144000,2.528900


[ 16, 9, 3, 648, 2592, 208, 165.1, 2.5289, 2.4714, 288000],


In [8]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  16 # 8 # G28 G18 #blocks
num_attention_heads = 9 # 4 # G16 G8
num_key_value_heads = 3 # 4 # G16 G1
hidden_size = num_attention_heads*68 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 208 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)
        
class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=32; B=28//2; N_step=144000*2; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(gradient_accumulation_steps=2, learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4714:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L16 att9 kv_heads3 hidden612 intermediate2448 head_dim208 T512


Step,Training Loss
144000,2.539300


[ 16, 9, 3, 612, 2448, 208, 151.7, 2.5393, 2.4714, 288000],


# L14 (Not used for figure because not explored enough)

In [ ]:
# L14 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)
[ 14, 8, 4, 704, 2816, 256, 179.3, 2.5348,+2.4772, 144000], # 27.3h 23.6G  minGemma-hidden_layers14-att_heads8-kv_heads4-hidden704-intermediate2816-head_dim256-T512--2025-05-30-00-01.pth

# L12

In [ ]:
# L12 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)

[ 12,12, 4, 792, 3168, 224, 198.4, 2.5497, 2.4816, 144000], # 30.1h 23.7G x4 TOO big
[ 12,12, 4, 768, 3072, 224, 189.7, 2.5357, 2.4775, 288000], # 29.5h 14.2G x4 TOO big
[ 12,12, 4, 768, 3072, 192, 180.3, 2.5403, 2.4789, 144000], # 26.3h 22.7G x4 TOO big
[ 12,12, 4, 720, 2880, 256, 181.7, 2.5502, 2.4782, 288000], # 30.6h 14.2G x4 TOO big
[ 12,12, 4, 720, 2880, 224, 172.9, 2.5349, 2.4769, 144000], # 28.0h 23.4G x4 TOO big
[ 12,12, 4, 720, 2880, 192, 164.0, 2.5436, 2.4798, 144000], # 25.9h 22.4G x4 TOO small
[ 12,12, 4, 672, 2688, 256, 165.0, 2.5538, 2.4769, 288000], # 29.7h       x4 TOO big
[ 12,12, 4, 672, 2688, 224, 156.7, 2.5463,+2.4745, 288000], # 28.1h 13.7G x4 TOO big
[ 12,12, 4, 672, 2688, 208, 152.6, 2.5440, 2.4782, 288000], # 27.5h 13.7G x4 TOO big
[+12,12, 4, 672, 2688, 192, 148.4, 2.5476,+2.4740, 288000], # 16.1h 13.4G x4 (MIN confirmed) minGemma-hidden_layers12-att_heads12-kv_heads4-hidden672-intermediate2688-head_dim192-T512--2025-07-17-17-31.pth
[ 12,12, 4, 672, 2688, 176, 144.3, 2.5397, 2.4802, 144000], # 24.4h 21.8G x4 TOO small
[ 12,12, 4, 624, 2496, 224, 141.2, 2.5430, 2.4754, 288000], # 26.2h       x4 TOO small
[ 12,12, 4, 624, 2496, 192, 133.5, 2.5531, 2.4776, 144000], # 23.5h 21.7G x4 TOO small

[ 12,10, 5, 760, 3040, 256, 191.5, 2.5654, 2.4816, 144000], # 28.0h 23.3G x4 TOO big  (if grad-acc 29.0h 14.1G = 3% time loss)
[ 12,10, 5, 720, 2880, 288, 185.6, 2.6100, 2.4854, 144000], # 30.6h 23.6G x4 TOO big
[ 12,10, 5, 720, 2880, 256, 177.3, 2.5397,+2.4772, 144000], # 27.5h 22.7G x4 (MIN-confirmed)
[ 12,10, 5, 720, 2880, 224, 169.0, 2.5616, 2.4795, 144000], # 26.1h 22.4G x4 TOO small
[ 12,10, 5, 680, 2720, 256, 163.5, 2.5533, 2.4785, 144000], # 27.0h 22.6G x4 TOO small
[ 12, 6, 3, 792, 3960, 256, 196.6, 2.5648, 2.4869, 144000], # 25.1h 23.4G x5 TOO small
[ 12, 6, 3, 792, 3168, 256, 174.0, 2.6050, 2.4918, 144000], # 23.7h 22.4G x4 TOO small

In [2]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  12 # 8 # G28 G18 #blocks
num_attention_heads =12 # 4 # G16 G8
num_key_value_heads = 4 # 4 # G16 G1
hidden_size = num_attention_heads*56 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 192 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)
        
class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=32; B=28//2; N_step=144000*2; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(gradient_accumulation_steps=2, learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4745:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L12 att12 kv_heads4 hidden672 intermediate2688 head_dim192 T512


Step,Training Loss
144000,2.547600


[ 12, 12, 4, 672, 2688, 192, 148.4, 2.5476, 2.4740, 288000],


# L9

In [ ]:
# L9 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)  # Too bad to explore
[ 9, 16, 4, 832, 3328, 224, 183.7, 2.5775, 2.4917, 144000], # 27.6h 23.1G x4 TOO big
[ 9, 16, 4, 768, 3072, 256, 173.2, 2.5584, 2.4844, 288000], # 28.3h 14.0G x4 TOO big  grad-acc
[+9, 16, 4, 768, 3072, 224, 164.3, 2.5599,+2.4840, 144000], # 26.0h       x4 (MIN confirmed)  minGemma-hidden_layers9-att_heads16-kv_heads4-hidden768-intermediate3072-head_dim224-T512--2025-06-16-01-35
[ 9, 16, 4, 768, 3072, 192, 155.5, 2.5609, 2.4849, 140000], # 23.8h 22.0G x4 TOO small
[ 9, 16, 4, 704, 2816, 224, 145.7, 2.5685, 2.4867, 144000], # 25.2h 22.1G x4 TOO small

# L8 (Not used for figure because not explored enough)

In [ ]:
# L8 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000) # Too bad to explore

[ 8,18, 3, 720, 3600, 256, 160.4, 2.5397, 2.4880, 144000], # 26.8h 23.1G x5 just size BASE-like
[ 8,18, 3, 720, 2880, 256, 147.9, 2.5505, 2.4889, 144000], # 26.0h 22.7G x4 just size (not bad)
[ 8,18, 3, 720, 2880, 224, 140.2, 2.5603, 2.4918, 144000], # 24.3h 22.2G x4 TOO small
[ 8,18, 3, 684, 2736, 192, 123.5, 2.5562, 2.4919, 144000], # 22.2h 21.0G x4 TOO small

[ 8,16, 4, 768, 3072, 160, 134.6, 2.5705, 2.4927, 144000], # 20.2h 20.6G
[ 8,16, 4, 704, 2816, 192, 126.3, 2.5621, 2.4890, 144000], # 21.0h

[+8,12, 4, 768, 3072, 288, 151.9, 2.5689,+2.4862, 144000], # x4 > BASEx5 23.7h 21.8G # minGemma-hidden_layers8-att_heads12-kv_heads4-hidden768-intermediate3072-head_dim288-T512--2025-05-18-16-42.pth
[ 8,12, 4, 768, 3072, 224, 139.3, 2.5652, 2.4894, 144000], # 20.2h TOO small
[ 8,12, 4, 768, 3072, 160, 126.7, 2.5780, 2.4908, 144000], # 17.8h TOO small


# H8 bad (hyperparameters explored for a small model)
[ 8, 8, 4, 800, 4800, 480, 206.2, 2.9463, 2.5474, 156000], # x6
[ 8, 8, 4, 800, 4000, 480, 190.8, 3.1778, 2.6310, 156000], # 29.9h
[ 8, 8, 4, 736, 3680, 256, 138.2, 2.6064, 2.4990, 150000], # 19.2h

[ 8, 8, 4, 800, 4800, 480, 206.2, 2.9332, 2.5479, 144000], # x6
[ 8, 8, 4, 800, 4000, 480, 190.8, 2.7179, 2.5262, 144000],
[ 8, 8, 4, 800, 4000, 320, 166.2, 2.6976, 2.5116, 144000],
[ 8, 8, 4, 800, 4000, 288, 161.3, 2.6004, 2.4940, 144000], # 21.7h
[ 8, 8, 4, 800, 4000, 256, 156.4, 2.6093, 2.4999, 144000], # 20.1h
[ 8, 8, 4, 800, 4000, 224, 151.5, 2.5853, 2.4950, 144000], # 19.3h
[ 8, 8, 4, 800, 4000, 192, 146.6, 2.5842, 2.4919, 144000], # 18.5h
[ 8, 8, 4, 800, 4000, 128, 136.8, 2.5824, 2.4957, 144000], # 15.2h
[ 8, 8, 4, 800, 3200, 480, 175.4, 3.1880, 2.6514, 144000], # 26.6h x4
[ 8, 8, 4, 768, 3840, 320, 156.6, 2.5828, 2.4907, 144000],
[ 8, 8, 4, 768, 3840, 288, 151.9, 2.6130, 2.4979, 144000], # 20.5h
[ 8, 8, 4, 768, 3840, 256, 147.2, 2.5794, 2.4910, 144000], # 18.9h
[ 8, 8, 4, 768, 3840, 224, 142.5, 2.5915, 2.4942, 144000], # 18.2h
[ 8, 8, 4, 768, 3840, 192, 137.8, 2.5860, 2.4932, 144000], # 17.1h
[ 8, 8, 4, 768, 3840, 128, 128.3, 2.5825, 2.4950, 144000], # 14.0h
[ 8, 8, 4, 736, 4416, 256, 151.3, 2.5802, 2.4930, 144000], # 19.4h x6
[ 8, 8, 4, 736, 3680, 320, 147.3, 2.5997, 2.4930, 144000], # 20.8h
[ 8, 8, 4, 736, 3680, 288, 142.8, 2.5939, 2.4913, 144000], # 20.1h 20.0G
[ 8, 8, 4, 736, 3680, 256, 138.2, 2.5676,+2.4863, 144000], # 18.4h "BASE" minGemma-hidden_layers8-att_heads8-kv_heads4-hidden736-intermediate3680-head_dim256-T512--2025-05-05-15-34.pth
[ 8, 8, 4, 736, 3680, 256, 138.2, 2.5036, 2.5066, 144000], # B36 33.6h 24.8G     (B24 < B28 > B32 > B36) N not optimized but overall B28 is always chosen 
[ 8, 8, 4, 736, 3680, 256, 138.2, 2.5295, 2.4965, 144000], # B32
[ 8, 8, 4, 736, 3680, 256, 138.2, 2.6282, 2.4908, 144000], # B24 16.0h 17.0G
[ 8, 8, 4, 736, 3680, 224, 133.7, 2.5715, 2.4895, 144000], # 17.8h 19.7G
[ 8, 8, 4, 736, 3680, 192, 129.2, 2.5700, 2.4921, 144000], # 16.7h
[ 8, 8, 4, 736, 3680, 128, 120.2, 2.6043, 2.4977, 144000], # 13.6h
[ 8, 8, 4, 736, 2944, 256, 125.2, 2.5705,+2.4886, 144000], # 17.6h 19.0G (BASE is x5 but x4 is enough for deep normal models?)
[ 8, 8, 4, 704, 3520, 288, 133.9, 2.5930, 2.4933, 144000], # 19.7h 19.8G
[ 8, 8, 4, 704, 3520, 256, 129.5, 2.5907, 2.4929, 144000], # 18.1h 19.4G
[ 8, 8, 4, 704, 3520, 224, 125.2, 2.5702, 2.4893, 144000], # 17.4h
[ 8, 8, 4, 672, 3360, 288, 125.2, 2.5839, 2.4894, 144000], # 19.2h
[ 8, 8, 4, 672, 3360, 256, 121.1, 2.5677, 2.4883, 144000], # 17.6h 19.0G
[ 8, 8, 4, 672, 3360, 224, 116.9, 2.5753, 2.4906, 144000], # 16.9h 18.9G

[ 8, 8, 4, 736, 3680, 256, 138.2, 2.6130, 2.4887, 138000], # x5
[ 8, 8, 4, 800, 4800, 480, 206.2, 3.2653, 2.6919, 132000], # x6 26h
[ 8, 8, 4, 800, 4000, 480, 190.8, 2.9327, 2.5308, 132000], # x5 25.7h 22.7G
[ 8, 8, 4, 736, 3680, 256, 138.2, 2.5720, 2.4998, 132000], # x5 B32